# Livrable 1

## Import des dépendances

In [122]:
# TensorFlow and tf.keras
import tensorflow as tf

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
import os
import pathlib
import time

## Chargement des données

In [123]:
#Paramètres

AUTOTUNE = tf.data.AUTOTUNE
current_dir = pathlib.Path().resolve()
DATASET_SIZE = len(list(current_dir.glob('data/*/*')))

batch_size = 32
img_height = 180
img_width = 180
num_classes = 6

In [137]:


full_dataset = tf.data.Dataset.list_files(str(current_dir / 'data/*/*'), shuffle=False)
full_dataset = full_dataset.shuffle(DATASET_SIZE, reshuffle_each_iteration=False)

train_size = int(0.7 * DATASET_SIZE)
val_size = int(0.15 * DATASET_SIZE)
test_size = int(0.15 * DATASET_SIZE)

train_dataset = full_dataset.take(train_size)
test_dataset = full_dataset.skip(train_size)
val_dataset = test_dataset.skip(val_size)
test_dataset = test_dataset.take(test_size)

print(DATASET_SIZE)
print(tf.data.experimental.cardinality(train_dataset).numpy())
print(tf.data.experimental.cardinality(test_dataset).numpy())
print(tf.data.experimental.cardinality(val_dataset).numpy())

41555
29088
6233
6234


In [139]:
class_names = np.array(sorted([item.name for item in current_dir.glob('data/*')]))
print(class_names)

['Dataset' 'Painting' 'Photo' 'Schematics' 'Sketch' 'Text']


In [140]:
def get_label(file_path):
  # Convert the path to a list of path components
  parts = tf.strings.split(file_path, os.path.sep)
  # The second to last is the class-directory
  one_hot = parts[-2] == class_names
  # Integer encode the label
  return tf.argmax(one_hot)

def decode_img(img):
  # Convert the compressed string to a 3D uint8 tensor
  img = tf.io.decode_jpeg(img, channels=3)
  # Resize the image to the desired size
  return tf.image.resize(img, [img_height, img_width])

def process_path(file_path):
  label = get_label(file_path)
  # Load the raw data from the file as a string
  img = tf.io.read_file(file_path)
  img = decode_img(img)
  return img, label

In [141]:
from tensorflow.python.data import AUTOTUNE

train_ds = train_dataset.map(process_path, num_parallel_calls=AUTOTUNE)
val_ds = val_dataset.map(process_path, num_parallel_calls=AUTOTUNE)
test_ds = test_dataset.map(process_path, num_parallel_calls=AUTOTUNE)

print(train_dataset)
print(val_dataset)
print(test_dataset)

<TakeDataset shapes: (), types: tf.string>
<SkipDataset shapes: (), types: tf.string>
<TakeDataset shapes: (), types: tf.string>


In [142]:
for image, label in train_ds.take(1):
  print("Image shape: ", image.numpy().shape)
  print("Label: ", label.numpy())

Image shape:  (180, 180, 3)
Label:  2


In [143]:
def configure_for_performance(ds):
  ds = ds.cache()
  ds = ds.shuffle(buffer_size=10000)
  ds = ds.batch(batch_size)
  ds = ds.prefetch(buffer_size=AUTOTUNE)
  return ds

train_ds = configure_for_performance(train_ds)
val_ds = configure_for_performance(val_ds)
test_ds = configure_for_performance(test_ds)

Standardisation des données

In [149]:
model = tf.keras.Sequential([
  tf.keras.layers.Rescaling(1./255),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Dropout(0.1),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Dropout(0.1),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Dropout(0.1),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(num_classes)
])

In [150]:
model.compile(
  optimizer='adam',
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

In [151]:
#Utilisation des rappels de point de contrôle

# Include the epoch in the file name (uses `str.format`)
checkpoint_path = "training/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

In [ ]:
# # Create a callback that saves the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True)

#Entraînement du modèle
model.fit(
  train_ds,
  callbacks=[cp_callback],
  validation_data=val_ds,
  epochs=3,
  shuffle=True
)

Epoch 1/3
909/909 [==============================] - ETA: 0s - loss: 0.5943 - accuracy: 0.7493
Epoch 00001: saving model to training\cp-0001.ckpt
909/909 [==============================] - 1242s 1s/step - loss: 0.5943 - accuracy: 0.7493 - val_loss: 0.4194 - val_accuracy: 0.8330
Epoch 2/3
901/909 [============================>.] - ETA: 11s - loss: 0.3801 - accuracy: 0.8511

In [ ]:
# checkpoint_dir = os.path.dirname(checkpoint_path)
# latest = tf.train.latest_checkpoint(checkpoint_dir)
# # Load the previously saved weights
# model.load_weights(latest)

## Validation des données

In [ ]:
unique_labels = np.unique(class_names)

def get_pred_label(prediction_probabilities):
  """
  Turns an array of prediction probabilities into a label.
  """

  return unique_labels[np.argmax(prediction_probabilities)]

def plot_pred(prediction_probabilities, labels, images, n=1):
  """
  View the prediction, ground truth and image for sample n
  """
  pred_prob, true_label, image = prediction_probabilities[n], labels[n], images[n]

  # Get the pred label
  pred_label = get_pred_label(pred_prob)

  # Plot image and remove ticks
  plt.imshow(image)
  plt.xticks([])
  plt.yticks([])

  # Change the color of the title depending on if the prediction is right or wrong
  if pred_label == true_label:
    color="green"
  else:
    color="red"

  # Change plot title to be predicted, probability of prediction and truth
  plt.title("{} {:2.0f}% {}".format(pred_label,
                                    np.max(pred_prob)*100,
                                    true_label),
                                    color=color)

def unbatchify(data):
  """
  Takes a batch dataset of (image, label) Tensors and returns separate arrays of images and labels
  """
  images = []
  labels = []

  # Loop through unbatched data
  for image, label in data.unbatch().as_numpy_iterator():
    images.append(image)
    labels.append(unique_labels[np.argmax(label)])

  return images,labels

In [ ]:
predictions = model.predict(val_ds)


In [ ]:
# plot_pred(prediction_probabilities=predictions,
#           labels=test_label,
#           images=test_img,
#           n=2)

In [ ]:
# history = model.history
#
# acc = history.history['accuracy']
# val_acc = history.history['val_accuracy']
#
# loss = history.history['loss']
# val_loss = history.history['val_loss']
#
# epochs_range = range(1)
#
# plt.figure(figsize=(16, 8))
# plt.subplot(1, 2, 1)
# plt.plot(epochs_range, acc, label='Training Accuracy')
# plt.plot(epochs_range, val_acc, label='Validation Accuracy')
# plt.legend(loc='lower right')
# plt.title('Training and Validation Accuracy')
#
# plt.subplot(1, 2, 2)
# plt.plot(epochs_range, loss, label='Training Loss')
# plt.plot(epochs_range, val_loss, label='Validation Loss')
# plt.legend(loc='upper right')
# plt.title('Training and Validation Loss')
# plt.show()

In [ ]:
# model.summary()
# print(test_ds)